# **1. Загрузка данных, разведочный анализ и чистка.**

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

In [3]:
!mkdir -p ~/.postgresql && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
    --output-document ~/.postgresql/root.crt && \
chmod 0600 ~/.postgresql/root.crt

--2023-02-28 17:23:48--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/root/.postgresql/root.crt’

/root/.postgresql/r 100%[===================>]   3.50K  --.-KB/s    in 0s      

2023-02-28 17:23:49 (431 MB/s) - ‘/root/.postgresql/root.crt’ saved [3579/3579]



In [4]:
!sudo apt update && sudo apt install --yes python3 python3-pip && \
!pip3 install psycopg2-binary

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Get:14 http://security.ubuntu.com

In [5]:
import psycopg2

conn = psycopg2.connect("""
    host=rc1b-70q90cunwkzx49mj.mdb.yandexcloud.net
    port=6432
    sslmode=verify-full
    dbname=db_clients
    user=mlhs_student
    password=mlhs_student
    target_session_attrs=read-write
""")

In [6]:
import pandas as pd

query = "SELECT * from public.credit_scoring"
data = pd.read_sql_query(query, conn)

conn.close()

In [7]:
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [8]:
data.shape

(150000, 11)

In [9]:
data.dtypes

SeriousDlqin2yrs                          int64
RevolvingUtilizationOfUnsecuredLines    float64
age                                       int64
NumberOfTime30-59DaysPastDueNotWorse      int64
DebtRatio                               float64
MonthlyIncome                           float64
NumberOfOpenCreditLinesAndLoans           int64
NumberOfTimes90DaysLate                   int64
NumberRealEstateLoansOrLines              int64
NumberOfTime60-89DaysPastDueNotWorse      int64
NumberOfDependents                      float64
dtype: object

In [10]:
data=data.drop_duplicates()

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149391 entries, 0 to 149999
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      149391 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  149391 non-null  float64
 2   age                                   149391 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  149391 non-null  int64  
 4   DebtRatio                             149391 non-null  float64
 5   MonthlyIncome                         120170 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       149391 non-null  int64  
 7   NumberOfTimes90DaysLate               149391 non-null  int64  
 8   NumberRealEstateLoansOrLines          149391 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  149391 non-null  int64  
 10  NumberOfDependents                    145563 non-null  float64
dtype

In [12]:
data.isna().sum()

SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29221
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3828
dtype: int64

In [13]:
data['NumberOfDependents'] = data['NumberOfDependents'].fillna(-1)

In [14]:
median = data['MonthlyIncome'].median()
print(median)
data['MonthlyIncome'].fillna(median, inplace=True)

5400.0


In [15]:
 data.SeriousDlqin2yrs. value_counts()

0    139382
1     10009
Name: SeriousDlqin2yrs, dtype: int64

In [16]:
data = data.drop(['NumberRealEstateLoansOrLines','NumberOfOpenCreditLinesAndLoans'],axis=1)

In [17]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
SeriousDlqin2yrs,1.000000,-0.001818,-0.115721,0.131517,-0.007719,-0.017262,0.122495,0.106257,0.047239
RevolvingUtilizationOfUnsecuredLines,-0.001818,1.000000,-0.005940,-0.001290,0.003946,0.006501,-0.001012,-0.000997,0.000662
age,-0.115721,-0.005940,1.000000,-0.055190,0.024191,0.027205,-0.052953,-0.048733,-0.223521
NumberOfTime30-59DaysPastDueNotWorse,0.131517,-0.001290,-0.055190,1.000000,-0.005912,-0.008380,0.980489,0.984535,-0.001338
DebtRatio,-0.007719,0.003946,0.024191,-0.005912,1.000000,-0.018105,-0.007823,-0.006963,-0.052226
MonthlyIncome,-0.017262,0.006501,0.027205,-0.008380,-0.018105,1.000000,-0.010689,-0.009328,0.066169
NumberOfTimes90DaysLate,0.122495,-0.001012,-0.052953,0.980489,-0.007823,-0.010689,1.000000,0.991409,-0.009533
NumberOfTime60-89DaysPastDueNotWorse,0.106257,-0.000997,-0.048733,0.984535,-0.006963,-0.009328,0.991409,1.000000,-0.010302
NumberOfDependents,0.047239,0.000662,-0.223521,-0.001338,-0.052226,0.066169,-0.009533,-0.010302,1.000000


In [18]:
X = data.drop('SeriousDlqin2yrs', axis=1) 
y = data['SeriousDlqin2yrs'] 

In [19]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=42)

In [20]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(Xtrain)

Xtrain = pd.DataFrame(ss.transform(Xtrain), columns=X.columns)
Xtest = pd.DataFrame(ss.transform(Xtest), columns=X.columns)

Xtrain.head()

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,-0.022107,1.201036,-0.101731,-0.173357,0.430620,-0.062544,-0.055916,0.251691
1,-0.024166,1.676242,-0.101731,-0.146922,-0.078595,-0.062544,-0.055916,-0.628410
2,-0.022649,-0.020920,-0.101731,-0.173017,-0.448934,-0.062544,-0.055916,-0.628410
3,-0.024210,-0.496125,-0.101731,-0.173392,-0.032303,-0.062544,-0.055916,0.251691
4,-0.022258,-0.835557,-0.101731,-0.173190,0.154101,-0.062544,-0.055916,-0.628410


# **2.Обучение логистической регрессии**

In [21]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression() 
model.fit(Xtrain, ytrain)

LogisticRegression()

In [22]:
pred_test = model.predict(Xtest)

In [23]:
pred_test

array([0, 0, 0, ..., 0, 0, 0])

In [24]:
accuracy_score(ytest, pred_test)

0.9341865695619578

In [25]:
y.value_counts()

0    139382
1     10009
Name: SeriousDlqin2yrs, dtype: int64

In [26]:
pred_test[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# **3.Валидация**

In [27]:
from sklearn.metrics import confusion_matrix

confusion_matrix(ytest, pred_test)

array([[34795,    62],
       [ 2396,    95]])

In [28]:
ytest.value_counts()

0    34857
1     2491
Name: SeriousDlqin2yrs, dtype: int64

In [29]:
from sklearn.metrics import recall_score

recall_score(ytest, pred_test)

0.0381372942593336

In [30]:
probs = model.predict_proba(Xtest)

probs[:10]

array([[0.97973046, 0.02026954],
       [0.9774043 , 0.0225957 ],
       [0.97035811, 0.02964189],
       [0.90596504, 0.09403496],
       [0.89233013, 0.10766987],
       [0.96808254, 0.03191746],
       [0.91338182, 0.08661818],
       [0.96757413, 0.03242587],
       [0.91210522, 0.08789478],
       [0.98322021, 0.01677979]])

In [34]:
probs_SeriousDlqin2yrs = probs[:,1]

classes = probs_SeriousDlqin2yrs > 0.4

In [35]:
confusion_matrix(ytest, classes)

array([[34755,   102],
       [ 2348,   143]])

In [36]:
ytest.value_counts()

0    34857
1     2491
Name: SeriousDlqin2yrs, dtype: int64

In [37]:
recall_score(ytest, classes)

0.057406663990365314

In [38]:
model.coef_, model.intercept_

(array([[-0.00576621, -0.4227669 ,  1.88294652, -0.0290203 , -0.38626713,
          1.7599248 , -3.47135448,  0.09436084]]), array([-2.84358807]))

In [39]:
import pandas as pd

coef_table = pd.DataFrame({'features' : list(X.columns), 'weights' : list(model.coef_[0])})

coef_table.sort_values(by='weights')

,features,weights
6,NumberOfTime60-89DaysPastDueNotWorse,-3.471354
1,age,-0.422767
4,MonthlyIncome,-0.386267
3,DebtRatio,-0.029020
0,RevolvingUtilizationOfUnsecuredLines,-0.005766
7,NumberOfDependents,0.094361
5,NumberOfTimes90DaysLate,1.759925
2,NumberOfTime30-59DaysPastDueNotWorse,1.882947


In [40]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

In [41]:
knn.fit(Xtrain, ytrain)

pred_test2 = knn.predict_proba(Xtest)[:,1]

classes2 = pred_test2 > 0.1

confusion_matrix(ytest, classes2)

array([[28251,  6606],
       [ 1085,  1406]])

# **4.СОХРАНЕНИЕ**

In [42]:
Xtest['predictions'] = probs_SeriousDlqin2yrs

Xtest[['predictions']].to_csv("PredictionsSeriousDlqin2yrs.csv", index=False)

In [43]:
import pickle 

with open('model.pickle', 'wb') as f:
    pickle.dump(model, f)